# 평가지표
- 정확도
- 모델 클래스는 직접 구현

In [39]:
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [40]:
DATA_PATH = "../data/pizza_steak_sushi/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

# 음식 분류 데이터셋
- 0 : 피자
- 1 : 스테이크
- 2 : 스시


In [41]:
train = pd.read_csv(f"{DATA_PATH}train.csv")
test = pd.read_csv(f"{DATA_PATH}test.csv")
train.shape , test.shape

((1649, 2), (1350, 2))

In [42]:
train.head()

,file_name,target
0,2104569.jpg,0
1,2038418.jpg,1
2,1919810.jpg,2
3,2557340.jpg,0
4,3621562.jpg,1


In [43]:
test.head()

,file_name,target
0,3777020.jpg,NaN
1,931356.jpg,NaN
2,2599817.jpg,NaN
3,1251166.jpg,NaN
4,1183595.jpg,NaN


데이터셋 클래스 ~ 테스트 데이터 예측 주석달기

# 1. 데이터셋 클래스 만들기

In [44]:
import cv2 # 이미지 처리를 위한 라이브러리

In [45]:
train_file = train["file_name"].to_numpy() # 파일명 추출 및 numpy 배열로 변환
test_file = test["file_name"].to_numpy() # 파일명 추출 및 numpy 배열로 변환

train_file.shape, test_file.shape # 파일명 배열의 shape 확인

((1649,), (1350,))

In [46]:
train_file[0] # 파일명 확인

'2104569.jpg'

In [47]:
train_target = train["target"].to_numpy() # 타겟 추출 및 numpy 배열로 변환

train_target.shape # 타겟 배열의 shape 확인

(1649,)

- transform 객체

In [48]:
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

resize = [224, 224]
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

train_list = [
    # 증강
    A.Resize(*resize),
    A.Normalize(mean, std),
    ToTensorV2()
]

train_transform = A.Compose(train_list)

test_list = [
    A.Resize(*resize),
    A.Normalize(mean, std),
    ToTensorV2()
]

test_transform = A.Compose(test_list)

In [49]:
class FoodsDataset(torch.utils.data.Dataset): # 데이터셋 클래스 정의
    def __init__(self, transform, x, y=None): # x: 파일명, y: 타겟(테스트 데이터 예측 시 정답 데이터가 없으므로 기본값 None 지정), resize: 이미지 크기
        self.transform = transform
        self.x = x # 파일명
        self.y = y # 타겟
        # self.resize = resize # 이미지 크기

    def __len__(self): # 데이터셋 길이 반환 메서드
        return len(self.x)

    def __getitem__(self, idx): # 데이터셋에서 특정 인덱스의 아이템 반환
        item = {} # 반환할 아이템을 담을 딕셔너리
        x = cv2.imread(f"{DATA_PATH}data/{self.x[idx]}") # 이미지 파일 읽기
        x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB) # 이미지 채널 순서 변경(BGR -> RGB)
        # x = cv2.resize(x, self.resize) # 이미지 크기 변경
        # x = x / 255 # 이미지 스케일링
        # item["x"] = torch.Tensor(x) # 이미지 텐서화
        item["x"] = self.transform(image=x)["image"]

        if self.y is not None: # 타겟이 존재할 경우
            item["y"] = torch.tensor(self.y[idx]) # 타겟 텐서화(다중 클래스 분류 문제이므로 텐서 함수 사용)

        return item # 아이템 반환

In [50]:
# 클래스 잘 작동하는지 확인
dataset = FoodsDataset(train_transform, train_file, train_target) # 데이터셋 객체 생성
dataloader = torch.utils.data.DataLoader(dataset, 2) # 데이터로더 생성
batch = next(iter(dataloader)) # 배치 생성
batch["x"].shape # 배치 shape 확인

torch.Size([2, 3, 224, 224])

# 모델 클래스 만들기

In [51]:
class Conv2dNet(torch.nn.Module): # 컨볼루션 신경망 클래스 정의
    def __init__(self, in_channels, out_channels, kernel_size): # in_channels: 입력 채널 수, out_channels: 출력 채널 수(커널 수), kernel_size: 커널 크기
        super().__init__() # 부모 클래스 초기화 메서드 실행
        self.seq = torch.nn.Sequential( # 순차적 계층 변수 생성
            torch.nn.Conv2d(in_channels, out_channels, kernel_size), # 컨볼루션 계층
            torch.nn.BatchNorm2d(out_channels), # 배치 정규화 계층
            torch.nn.PReLU(), # 활성화 함수
            torch.nn.MaxPool2d(2), # 맥스 풀링 계층
        )

    def forward(self, x): # 생성한 순차적 계층 실행 메서드
        return self.seq(x) # 순차적 계층에 입력값 x를 순차적으로 통과시킨 결과 반환

In [52]:
class LinearNet(torch.nn.Module): # 완전 연결 신경망 클래스 정의
    def __init__(self, in_channels, out_channels): # in_channels: 입력 채널 수, out_channels: 출력 채널 수(커널 수)
        super().__init__() # 부모 클래스 초기화 메서드 실행
        self.seq = torch.nn.Sequential( # 순차적 계층 변수 생성
            torch.nn.Dropout(0.2), # 드롭아웃 계층
            torch.nn.Linear(in_channels, out_channels), # 완전 연결 계층
            torch.nn.BatchNorm1d(out_channels), # 배치 정규화 계층
            torch.nn.PReLU(), # 활성화 함수
        )

    def forward(self, x): # 생성한 순차적 계층 실행 메서드
        return self.seq(x) # 순차적 계층에 입력값 x를 순차적으로 통과시킨 결과 반환

In [53]:
class Net(torch.nn.Module): # 전체 신경망 클래스 정의
    def __init__(self, in_channels=3, out_channels=16, kernel_size=3): # in_channels: 입력 채널 수, out_channels: 출력 채널 수(커널 수), kernel_size: 커널 크기
        super().__init__() # 부모 클래스 초기화 메서드 실행
        self.seq = torch.nn.Sequential( # 순차적 계층 변수 생성
            Conv2dNet(in_channels, out_channels, kernel_size), # 컨볼루션 신경망
            Conv2dNet(out_channels, out_channels * 2, kernel_size), # 컨볼루션 신경망
            Conv2dNet(out_channels * 2, out_channels * 4, kernel_size), # 컨볼루션 신경망
            Conv2dNet(out_channels * 4, out_channels * 8, kernel_size), # 컨볼루션 신경망

            torch.nn.AdaptiveMaxPool2d(1), # 글로벌 맥스 풀링 계층
            torch.nn.Flatten(), # 텐서 펼치기
            LinearNet(out_channels * 8, out_channels * 4), # 완전 연결 신경망
            LinearNet(out_channels * 4, out_channels * 2), # 완전 연결 신경망
            torch.nn.Dropout(0.2), # 드롭아웃 계층
            torch.nn.Linear(out_channels * 2, 3), # 완전 연결 신경망(정답 데이터 클래스 개수가 3개이므로 출력 채널 수 3으로 설정)
        )

    def forward(self, x): # 생성한 순차적 계층 실행 메서드
        return self.seq(x.permute(0, 3, 1, 2)) # 순차적 계층에 입력값 x를 순차적으로 통과시킨 결과 반환(입력값 x의 채널 순서 변경)

In [54]:
import timm

model_name = 'efficientnet_b0.ra4_e3600_r224_in1k'

class Net(torch.nn.Module):
    def __init__(self, model_name, num_classes):
        super().__init__()
        self.pre_model = timm.create_model(model_name, pretrained=True, num_classes=num_classes)
    
    def forward(self, x):
        return self.pre_model(x)

In [55]:
model = Net(model_name, 1)
model(batch["x"])

tensor([[-3.7344],
        [ 4.4264]], grad_fn=<AddmmBackward0>)

# 2. 학습 loop 함수 만들기

In [56]:
def train_loop(dataloader, model, loss_function, optimizer, device): # 학습 함수 정의
    epoch_loss = 0 # 에폭 손실값 변수 초기화
    model.train() # 모델 학습 상태로 설정

    for batch in dataloader: # 데이터로더에서 미니 배치를 하나씩 꺼내 반복
        pred = model(batch["x"].to(device)) # 모델에 입력값 x를 넣어 예측값 계산, to 메서드로 연산을 수행할 디바이스로 입력값 x 이동
        loss = loss_function(pred, batch["y"].to(device)) # 예측값과 정답 데이터로 손실값 계산, to 메서드로 연산을 수행할 디바이스로 정답 데이터 이동

        optimizer.zero_grad() # 기울기 초기화
        loss.backward() # 손실값 역전파
        optimizer.step() # 옵티마이저 가중치 업데이트

        epoch_loss += loss.item() # 손실값 파이썬 숫자로 변환하여 에폭 손실값에 더함

    epoch_loss /= len(dataloader) # 에폭 손실값을 데이터로더 길이로 나누어 평균 손실값 계산
    return epoch_loss # 에폭 손실값 반환

# 3. 테스트 loop 함수 만들기
- 데이터 예측 기능 및 검증데이터 손실값 반환하는 기능

In [57]:
@torch.no_grad() # 기울기 계산하지 않도록 데코레이터 설정
def test_loop(dataloader, model, loss_function, device): # 테스트 함수 정의
    epoch_loss = 0 # 에폭 손실값 변수 초기화
    model.eval() # 모델 평가 상태로 설정
    act = torch.nn.Softmax(dim=1) # 활성화 함수 설정(다중 클래스 분류 문제이므로 소프트맥스 함수 사용)
    pred_list = [] # 예측값을 담을 리스트 초기화
    
    for batch in dataloader: # 데이터로더에서 미니 배치를 하나씩 꺼내 반복
        pred = model(batch["x"].to(device)) # 모델에 입력값 x를 넣어 예측값 계산, to 메서드로 연산을 수행할 디바이스로 입력값 x 이동
        if batch.get("y") is not None: # 정답 데이터가 존재할 경우(검증 데이터)
            loss = loss_function(pred, batch["y"].to(device)) # 예측값과 정답 데이터로 손실값 계산, to 메서드로 연산을 수행할 디바이스로 정답 데이터 이동
            epoch_loss += loss.item() # 손실값 파이썬 숫자로 변환하여 에폭 손실값에 더함

        pred = act(pred) # 활성화 함수 적용
        pred = pred.to("cpu").numpy() # 텐서를 cpu 메모리로 이동한 후 넘파이 배열로 변환
        pred_list.append(pred) # 예측값 리스트에 추가

    pred = np.concatenate(pred_list) # 예측값 리스트를 연결
    epoch_loss /= len(dataloader) # 에폭 손실값을 데이터로더 길이로 나누어 평균 손실값 계산

    return epoch_loss, pred # 에폭 손실값과 예측값 반환

# 4. 학습하기


In [58]:
n_splits = 5 # 폴드 수 설정
batch_size = 32 # 배치 크기 설정
epochs = 100 # 에폭 수 설정
loss_function = torch.nn.CrossEntropyLoss() # 손실 함수 설정(다중 클래스 분류 문제이므로 크로스 엔트로피 손실 함수 사용)

In [59]:
from sklearn.model_selection import KFold # KFold 클래스 불러오기
from sklearn.metrics import accuracy_score # 정확도 계산 함수 불러오기

cv = KFold(n_splits, shuffle=True, random_state=SEED) # KFold 객체 생성

In [62]:
is_holdout = False # 홀드아웃 검증 사용 여부 설정
reset_seeds(SEED) # 시드 고정
score_list = [] # 점수를 담을 리스트 초기화

for i, (tri, vai) in enumerate(cv.split(train_file)): # KFold 객체를 통해 학습 데이터 인덱스와 검증 데이터 인덱스를 생성하고 인덱스와 함께 반복
    model = Net(model_name, 3).to(device) # 모델 생성 및 디바이스 설정
    optimizer = torch.optim.Adam(model.parameters()) # 옵티마이저 생성 및 모델 파라미터 등록

    train_dataset = FoodsDataset(train_transform, train_file[tri], train_target[tri]) # 학습 데이터셋 객체 생성
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True) # 학습 데이터로더 객체 생성

    valid_dataset = FoodsDataset(test_transform, train_file[vai], train_target[vai]) # 검증 데이터셋 객체 생성
    valid_dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False) # 검증 데이터로더 객체 생성

    best_score = 0 # 최고 점수(정확도) 변수 초기화
    best_loss = np.inf # 최저 손실값(검증 손실) 변수 초기화
    patience = 0 # 조기 종료 변수 초기화

    for epoch in tqdm(range(epochs)): # 에폭 수 만큼 반복
        train_loss = train_loop(train_dataloader, model, loss_function, optimizer, device) # 학습 함수 실행
        valid_loss, pred = test_loop(valid_dataloader, model, loss_function, device) # 검증 함수 실행
        pred = np.argmax(pred, axis=1) # 예측값 중 가장 큰 값의 인덱스를 정답으로 설정
        score = accuracy_score(train_target[vai], pred) # 검증 데이터 정답과 예측값으로 정확도 계산

        print(f"train_loss: {train_loss: .4f}, valid_loss: {valid_loss: .4f}, score: {score: .4f}") # 학습 손실값, 검증 손실값, 정확도 출력
        patience += 1 # 조기 종료 변수 1 추가
        if score > best_score or best_loss > valid_loss: # 정확도가 높아지거나 검증 손실값이 낮아질 경우
            patience = 0 # 조기 종료 변수 초기화
            best_score = score # 최고 점수(정확도) 갱신
            best_loss = valid_loss # 최저 손실값(검증 손실) 갱신
            torch.save(model.state_dict(), f"../output/model_{i}.pt") # 모델 저장

        if patience == 10: # 조기 종료 조건(10번 연속으로 최고 점수, 최저 손실값 갱신이 일어나지 않을 경우)
            break # 학습 종료

    print(f"Fold-{i} Best Loss: {best_loss: .4f}, Best Acc: {best_score: .4f}") # 폴드별 최고 정확도 출력
    score_list.append(best_score) # 폴드별 최고 정확도를 리스트에 추가

    if is_holdout: # 홀드아웃 검증 사용할 경우
        break # 반복문 종료

  0%|          | 0/100 [00:00<?, ?it/s]

train_loss:  1.8998, valid_loss:  19653.2274, score:  0.5758
train_loss:  0.7126, valid_loss:  553.9701, score:  0.4121
train_loss:  0.4282, valid_loss:  9.6122, score:  0.7788
train_loss:  0.2691, valid_loss:  0.5452, score:  0.8364
train_loss:  0.3216, valid_loss:  0.4694, score:  0.8182
train_loss:  0.1586, valid_loss:  0.4120, score:  0.8576
train_loss:  0.1244, valid_loss:  0.4946, score:  0.8485
train_loss:  0.0770, valid_loss:  0.5466, score:  0.8667
train_loss:  0.0429, valid_loss:  0.4323, score:  0.8848
train_loss:  0.0545, valid_loss:  0.6611, score:  0.8394
train_loss:  0.2031, valid_loss:  1.3517, score:  0.5667
train_loss:  0.2307, valid_loss:  0.5718, score:  0.8121
train_loss:  0.1525, valid_loss:  0.3548, score:  0.8758
train_loss:  0.0711, valid_loss:  0.5688, score:  0.8727
train_loss:  0.0186, valid_loss:  0.5333, score:  0.8939
train_loss:  0.0305, valid_loss:  0.6834, score:  0.8606
train_loss:  0.0923, valid_loss:  0.5913, score:  0.8455
train_loss:  0.1397, vali

  0%|          | 0/100 [00:00<?, ?it/s]

train_loss:  2.0985, valid_loss:  5.4793, score:  0.3636
train_loss:  0.5619, valid_loss:  0.8471, score:  0.7333
train_loss:  0.2346, valid_loss:  0.8880, score:  0.7485
train_loss:  0.2129, valid_loss:  0.5612, score:  0.8152
train_loss:  0.1330, valid_loss:  0.5222, score:  0.8515
train_loss:  0.0883, valid_loss:  0.6405, score:  0.8364
train_loss:  0.1206, valid_loss:  0.6556, score:  0.8152
train_loss:  0.1962, valid_loss:  0.8335, score:  0.7909
train_loss:  0.1521, valid_loss:  0.5134, score:  0.8303
train_loss:  0.0651, valid_loss:  0.4693, score:  0.8485
train_loss:  0.0909, valid_loss:  0.5717, score:  0.8091
train_loss:  0.2634, valid_loss:  0.6686, score:  0.7727
train_loss:  0.0829, valid_loss:  0.4658, score:  0.8455
train_loss:  0.1462, valid_loss:  0.4341, score:  0.8394
train_loss:  0.1146, valid_loss:  0.4578, score:  0.8424
train_loss:  0.0784, valid_loss:  0.5564, score:  0.8030
train_loss:  0.0560, valid_loss:  0.4780, score:  0.8788
train_loss:  0.0615, valid_loss

  0%|          | 0/100 [00:00<?, ?it/s]

train_loss:  2.3106, valid_loss:  46.4458, score:  0.3697
train_loss:  1.2675, valid_loss:  0.9752, score:  0.7061
train_loss:  0.4044, valid_loss:  0.6042, score:  0.8152
train_loss:  0.3114, valid_loss:  0.7279, score:  0.7697
train_loss:  0.1705, valid_loss:  0.7038, score:  0.8152
train_loss:  0.1454, valid_loss:  0.5585, score:  0.8303
train_loss:  0.1313, valid_loss:  0.5284, score:  0.8485
train_loss:  0.1001, valid_loss:  0.6191, score:  0.8576
train_loss:  0.1678, valid_loss:  0.6293, score:  0.8273
train_loss:  0.1919, valid_loss:  0.9777, score:  0.8152
train_loss:  0.0904, valid_loss:  546.4352, score:  0.8000
train_loss:  0.0972, valid_loss:  0.6737, score:  0.8303
train_loss:  0.0866, valid_loss:  2295.0257, score:  0.8212
train_loss:  0.0914, valid_loss:  236.5361, score:  0.8121
train_loss:  0.1924, valid_loss:  1257.3697, score:  0.8636
train_loss:  0.0656, valid_loss:  1.5602, score:  0.8697
train_loss:  0.0344, valid_loss:  887.6749, score:  0.8515
train_loss:  0.071

  0%|          | 0/100 [00:00<?, ?it/s]

train_loss:  2.5447, valid_loss:  5.3793, score:  0.3242
train_loss:  0.9257, valid_loss:  1.7061, score:  0.5394
train_loss:  0.3248, valid_loss:  0.6172, score:  0.7848
train_loss:  0.2068, valid_loss:  0.4697, score:  0.8394
train_loss:  0.1473, valid_loss:  0.4699, score:  0.8727
train_loss:  0.1577, valid_loss:  0.6060, score:  0.8303
train_loss:  0.1805, valid_loss:  0.5728, score:  0.8121
train_loss:  0.1600, valid_loss:  0.3152, score:  0.8758
train_loss:  0.0753, valid_loss:  0.4077, score:  0.8909
train_loss:  0.0635, valid_loss:  0.5982, score:  0.8697
train_loss:  0.0663, valid_loss:  0.5955, score:  0.8485
train_loss:  0.1249, valid_loss:  0.4362, score:  0.8515
train_loss:  0.0564, valid_loss:  0.3518, score:  0.8848
train_loss:  0.1861, valid_loss:  0.8462, score:  0.7788
train_loss:  0.1468, valid_loss:  0.5966, score:  0.8152
train_loss:  0.0770, valid_loss:  0.4698, score:  0.8727
train_loss:  0.0961, valid_loss:  0.3718, score:  0.8848
train_loss:  0.0428, valid_loss

  0%|          | 0/100 [00:00<?, ?it/s]

train_loss:  2.5518, valid_loss:  10216030.4194, score:  0.3435
train_loss:  0.8903, valid_loss:  12029.4766, score:  0.7416
train_loss:  0.5280, valid_loss:  2868.5320, score:  0.8116
train_loss:  0.3570, valid_loss:  1370.3406, score:  0.8511
train_loss:  0.2418, valid_loss:  21.5321, score:  0.8267
train_loss:  0.1684, valid_loss:  0.8706, score:  0.7964
train_loss:  0.1876, valid_loss:  25.9204, score:  0.7872
train_loss:  0.1319, valid_loss:  0.4833, score:  0.8176
train_loss:  0.0842, valid_loss:  5.3182, score:  0.8511
train_loss:  0.0864, valid_loss:  237.2914, score:  0.7842
train_loss:  0.1849, valid_loss:  0.4803, score:  0.8602
train_loss:  0.0762, valid_loss:  25892.6332, score:  0.8116
train_loss:  0.0608, valid_loss:  0.4070, score:  0.8967
train_loss:  0.0900, valid_loss:  0.4471, score:  0.8571
train_loss:  0.1154, valid_loss:  0.3980, score:  0.8784
train_loss:  0.0560, valid_loss:  0.4942, score:  0.8754
train_loss:  0.1563, valid_loss:  2.5848, score:  0.8024
train_

In [63]:
np.mean(score_list) # 평균 정확도 계산

0.8957041540020263

# 5. 테스트 데이터 예측

In [64]:
test_dataset = FoodsDataset(test_transform, test_file) # 테스트 데이터셋 객체 생성
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False) # 테스트 데이터로더 객체 생성

In [65]:
pred_list = [] # 예측값을 담을 리스트 초기화
for i in range(n_splits): # 폴드 수 만큼 반복
    model = Net(model_name, 3).to(device) # 모델 생성 및 디바이스 설정
    model_params = torch.load(f"../output/model_{i}.pt", weights_only=True) # 모델 가중치 불러오기
    model.load_state_dict(model_params) # 모델 가중치 업데이트
    _, pred = test_loop(test_dataloader, model, loss_function, device) # 예측을 위한 테스트 함수 실행
    pred_list.append(pred) # 예측값 리스트에 추가

pred = np.mean(pred_list, axis=0) # 예측값 평균 계산
pred = np.argmax(pred, axis=1) # 예측값 중 가장 큰 값의 인덱스를 정답으로 설정
pred # 예측값 출력

array([1, 1, 1, ..., 1, 2, 1], dtype=int64)

# 6. 칸스 사이트의 컴피티션 페이지에 제출하여 점수 확인해보세요.

In [66]:
pd.DataFrame(pred, columns=["target"]).to_csv("../output/foods_classification_CNN.csv", index=False)